In [ ]:
import json
from lxml import html
from urllib.request import Request, urlopen
import unicodecsv as csv

In [ ]:
# create a function that takes postal code as inputs

def create_url(pcode):
    url = "https://www.zillow.com/homes/{0}".format(pcode)
    return url

In [ ]:
# data cleaning from jason

def clean(text):
    # clean up text string
    if text:
        return ' '.join(' '.join(text).split())
    return None

def get_data_from_json(raw_json_data):
    cleaned_data = clean(raw_json_data).replace('<!--', "").replace("-->", "")
    properties_list = []

    try:
        json_data = json.loads(cleaned_data) # parse cleaned_data
        search_results = json_data.get('cat1').get('searchResults').get('listResults', [])

        # find all the attributes from webpage source
        for properties in search_results:
            property_info = properties.get('hdpData', {}).get('homeInfo')
            address = property_info.get('streetAddress')
            city = property_info.get('city')
            state = property_info.get('state')
            postal_code = property_info.get('zipcode')
            lat = property_info.get('latitude')
            lon = property_info.get('longitude')
            days = property_info.get('daysOnZillow')
            price = properties.get('price')
            bedrooms = properties.get('beds')
            bathrooms = properties.get('baths')
            area = properties.get('area')
            info = f'{bedrooms} bds, {bathrooms} ba ,{area} sqft'
            broker = properties.get('brokerName')
            property_url = properties.get('detailUrl')
            title = properties.get('statusText')

            # store data
            data = {'address': address,
                    'city': city,
                    'state': state,
                    'postal_code': postal_code,
                    'lat': lat,
                    'lon': lon,
                    'days': days,
                    'price': price,
                    'facts and features': info,
                    'real estate provider': broker,
                    'url': property_url,
                    'title': title}

            # list of all listings
            properties_list.append(data)
        return properties_list

    except ValueError:
        print("Invalid json")
        return None

In [ ]:
def web_scrape(pcode):
    
    # download the website
    url = create_url(pcode)
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) 
    webpage = urlopen(req).read()
    
    # parsing 
    parser = html.fromstring(webpage)
    search_results = parser.xpath("//div[@id='search-results']//article")
    raw_json_data = parser.xpath('//script[@data-zrr-shared-data-key="mobileSearchPageStore"]//text()')
    
    # reformat data 
    properties_list = get_data_from_json(raw_json_data)
    return properties_list

In [ ]:
# exporting
def write_data_to_csv(data):
    with open("mls_data/all_properties.csv", 'wb') as csvfile:
        fieldnames = [
            'title', 
            'address', 
            'city', 
            'state', 
            'postal_code', 
            'lat',
            'lon',
            'price', 
            'days',
            'facts and features', 
            'real estate provider', 
            'url']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [ ]:
# input 
pcode_lst = ["M4V"]

# append zipcodes 
final_list = []
for pcode in pcode_lst:
    print ("Fetching data for %s" % (pcode))
    
    # main function 
    scraped_data = web_scrape(pcode)
    for data in scraped_data:
        final_list.append(data)
    print ("Done for %s." % (pcode))  

# export data
write_data_to_csv(final_list)

### Toronto
get more pcodes: 
https://worldpostalcode.com/canada/ontario/toronto

In [ ]:
pcode_lst = ["M4V", "M4T", "M5R", "M5P", "M5N", "M4N", "M4P", "M4R", "M4S", "M3K", "M5W", "M7Y", "M8V",
               "M5S", "M5B", "M5X", "M5V", "M4W", "M4X", "M4Y", "M5A", "M5C", "M5T", "M5E", "M5G", "M5H", "M5J",  
               "M5K", "M5L", "M6G", "M4E", "M4M", "M4L", "M4K", "M4J", "M6H", "M6J", "M6K", "M6P", "M6R", "M6S",]